# Import all the necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from keras import models 
import warnings
from keras.models import load_model
from torchvision import datasets
import torchvision
import torch
import cv2
from torchvision.transforms import transforms
import torchvision.models as models
from PIL import ImageFile
from PIL import Image
from sklearn.model_selection import train_test_split
ImageFile.LOAD_TRUNCATED_IMAGES = True
plt.ion()
# Ignore all warnings
warnings.filterwarnings('ignore')

# Setting GPU Memory

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])  # Set memory limit to 4GB

# Checking GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
IMG = 255
IMG_SIZE = [IMG,IMG]
numofClasses = 3
batchSize = 16
EPOCHS = 20
PATIENCE = 5

# Collect the Data from Any Possible Resources
Dataset Collection
Objective: Obtain a labeled dataset that includes images of fire, smoke, and non-fire scenarios.
Sources: The dataset used in this project is the FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET.
Details: The dataset is structured in a directory format, with subfolders representing each of the three classes: fire, smoke, and non fire.
Action: Ensure the images are in a readable format (e.g., JPEG, PNG).

In [ ]:
train_set = '/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/'
valid_set = '/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/'

# Data Preprocessing
Task 2.1: Corrupted Image Removal
Objective: Identify and remove corrupted images that do not conform to the JFIF format.
Code: The code iterates over the training and testing directories, checking for corrupted images, and deletes them.

In [ ]:
num_skipped = 0
for folder_name in ('Neutral','Fire', 'Smoke'):
    folder_path = os.path.join("/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")

num_skipped = 0
for folder_name in ('Neutral','Fire', 'Smoke'):
    folder_path = os.path.join("/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")



# Model Selection, Training, Predicting, and Assessment
Task 6.1: Model Architecture
Objective: Select and build a model for classification using CNNs.
Details: A simple CNN model is chosen with multiple convolutional layers followed by max-pooling layers. Dropout layers are used to reduce overfitting.

In [ ]:
# Build the Model:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(96,(3,3),activation='relu',input_shape = [IMG,IMG,3]),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(256,activation='relu'),

    tf.keras.layers.Dense(numofClasses, activation='softmax')



])

print(model.summary())

# Choose the Metrics for the Model Evaluation
Task 5.1: Model Evaluation Metrics
Objective: Choose appropriate metrics for evaluating the model's performance.
Metrics Chosen: Accuracy is used as the primary evaluation metric because this is a classification task. Additionally, a confusion matrix can be used for performance analysis (accuracy, precision, recall, F1-score).

In [ ]:
# Set desired learning rate
learning_rate = 0.0001  # learning rate

# Create the Adam optimizer with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Now compile the model with the optimizer, loss function, and metrics
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',  # Change based on your problem
              metrics=['accuracy'])  # or other metrics you want to track

In [ ]:
#model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Data Augmentation
Objective: Increase the diversity of the training data to avoid overfitting.
Code: Use Keras' ImageDataGenerator for augmentation techniques such as rotation, zoom, shift, shear, and horizontal flip.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. /255,
                                   rotation_range = 20,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.1
                                   )

test_datagen = ImageDataGenerator(rescale=1. /255,validation_split=0.1)

# Image Resizing and Normalization
Objective: Preprocess the images by resizing and normalizing the pixel values to a range of [0,1].
Code: Images are resized to 255x255 pixels and normalized.

In [ ]:
training_set = train_datagen.flow_from_directory(train_set,
                                                 shuffle = True,
                                                 target_size = IMG_SIZE,
                                                 batch_size=batchSize,
                                                 class_mode = 'categorical',
                                                 color_mode='rgb'
                                                 )

testing_set = train_datagen.flow_from_directory(valid_set,
                                                 shuffle = False,
                                                 target_size = IMG_SIZE,
                                                 batch_size=batchSize,
                                                 class_mode = 'categorical',
                                                 color_mode='rgb')

In [ ]:
class_names = training_set.class_indices
class_names

In [ ]:
stepsPerEpochs = int(np.ceil(training_set.samples / batchSize))
validationsteps = int(np.ceil(testing_set.samples / batchSize))

In [ ]:
stepsPerEpochs

In [ ]:
best_model_file = "C:/Users/sumit/Downloads/Compressed/fire_and_smoke_model_final.keras"
bestModel = ModelCheckpoint(best_model_file, monitor='val_accuracy',verbose=1,save_best_only=True)

In [ ]:
steps_per_epoch = int(stepsPerEpochs)
validation_steps = int(validationsteps)
epochs = int(EPOCHS)

# Model Training
Objective: Train the model using the training dataset.
Details: Training is done using the fit method. Early stopping and model checkpointing are used to prevent overfitting and save the best model.

In [ ]:
history = model.fit(
    training_set,
    validation_data = testing_set,
    epochs = epochs,
    steps_per_epoch = stepsPerEpochs,
    validation_steps = validationsteps,
    verbose = 1,
    callbacks = [bestModel]
)

# Model Assessment
Objective: Evaluate the model's performance on the test set.

In [ ]:
valResults = model.evaluate(testing_set)
print(valResults)
print(model.metrics_names)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

actualEpochs = range(len(acc))
print("Actual Epochs : "+ str(actualEpochs))

In [ ]:
plt.plot(actualEpochs,acc , 'r' , label = "Training accuracy")
plt.plot(actualEpochs, val_acc , 'b' , label = "Validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and validation accuracy")
plt.show()

In [ ]:
model.save("fire_and_smoke_model_final.keras")

# Hyperparameter Tuning/Model Improvement
 Hyperparameter Optimization Objective: Optimize the model by tuning hyperparameters such as the number of filters, learning rate, dropout rate, etc. Tools Used: Keras Tuner is used for hyperparameter optimization via the RandomSearch method.

In [ ]:
# Build the Model:

def build_model(hp):
    filter_layer1 = hp.Int("filters_layers1",min_value=32, max_value=256, step=32)
    filter_layer2 = hp.Int("filters_layers2",min_value=32, max_value=256, step=32)
    filter_layer3 = hp.Int("filters_layers3",min_value=32, max_value=256, step=32)
    filter_layer4 = hp.Int("filters_layers4",min_value=32, max_value=256, step=32)

    hp_learning_rate = hp.Choice('learning_rate',values = [1e-2, 1e-3, 1e-4])
    hp_optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate)
    hp_dropout = hp.Choice('drop_out', values = [0.3,0.5])
    hp_last_dense_layer = hp.Int('last_dense_layer', min_value = 128, max_value = 786, step = 64)




    model1 = tf.keras.models.Sequential([

        tf.keras.layers.Conv2D(filter_layer1,kernel_size = (3,3),activation='relu',input_shape = [IMG,IMG,3]),
        tf.keras.layers.MaxPool2D(2,2),

        tf.keras.layers.Conv2D(filter_layer2,kernel_size = (3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),

        tf.keras.layers.Conv2D(filter_layer3,kernel_size = (3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),

        tf.keras.layers.Conv2D(filter_layer4,kernel_size = (3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(hp_dropout),

        tf.keras.layers.Dense(hp_last_dense_layer,activation='relu'),

        tf.keras.layers.Dense(numofClasses, activation='softmax')



    ])

    model1.compile(loss='categorical_crossentropy',optimizer=hp_optimizer,metrics=['accuracy'])



    return model1


In [ ]:
best_model_file = "C:/Users/sumit/Downloads/Compressed/fire_and_smoke_model_eran_hyper_peram_1.keras"
# bestModel = ModelCheckpoint(best_model_file, monitor='val_accuracy',verbose=1,save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = PATIENCE)

In [ ]:
# Keras Tunner
import keras_tuner
from keras_tuner import RandomSearch

tunner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 3,
    executions_per_trial = 12,
    directory = 'C:/Users/sumit/Downloads/Compressed',
    project_name = 'FireSmoke_dete_Random_search_HP',
    overwrite = True)

In [ ]:
print(f"Training set: {training_set}")
print(f"Testing set: {testing_set}")
print(f"EPOCHS: {EPOCHS}")
print(f"Batch size: {batchSize}")
print(f"Steps per epoch: {stepsPerEpochs}")
print(f"Validation steps: {validationsteps}")

# Now, call the search method
tunner.search(
    training_set,
    validation_data=testing_set,
    epochs=EPOCHS,
    batch_size=batchSize,
    callbacks=[stop_early],
    steps_per_epoch=stepsPerEpochs,
    validation_steps=validationsteps
)

# Best Hyperparameter Selection
Objective: Retrieve and use the best hyperparameters found through the tuning process.

In [ ]:
#best perameter
best_hp = tunner.get_best_hyperparameters()[0].values
print("==========================")
print("Best model parameters :")
print(best_hp)
print("==========================")
print("  ")

# Best Model Selection
Objective: Retrieve and use the best model found through the tuning process.

In [ ]:
#best Model
model = tunner.get_best_models(num_models = 1)[0]
print("==========================")
print("Best model :")
print(model.summary())
print("==========================")
print("  ")

# for better accuracy the model I have used transfer learning technique. Architecture used here is ResNet.
I have achieved validation accuracy of 93% using ResNet.

In [ ]:
batch_size = 64
num_workers = 0

transform = transforms.Compose([
    transforms.Resize(size=(256,256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_set = '/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train'
valid_set = '/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test'


train_data = datasets.ImageFolder(train_set, transform=transform)
valid_data = datasets.ImageFolder(valid_set, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = {
    'train': train_loader,
    'valid': valid_loader
}

In [ ]:
class_names = ['Neutral','Fire', 'Smoke']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.resnet50(pretrained=True)

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Sequential(torch.nn.Linear(2048,128),
                                      torch.nn.ReLU(),
                                       torch.nn.Linear(128,3),
                                       torch.nn.Softmax()
                                      )

for param in model.fc.parameters():
    param.requires_grad = True
    
if use_cuda:
    model_transfer = model.cuda()
    
model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_transfer.fc.parameters(), lr=0.0005)

In [ ]:
n_epochs = 100

train_accuracy_list = []
train_loss_list = []
valid_accuracy_list = []
valid_loss_list = []

def train(n_epochs, loader, model, optimizer, criterion, use_cuda, save_path):
    
    valid_loss_min = np.inf
       
    for epoch in range(1, (n_epochs+1)):
        
        train_loss = 0.0
        valid_loss = 0.0
        train_acc = 0.0
        valid_acc = 0.0
        
        model.train()
        
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            _, preds = torch.max(output, 1)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_acc = train_acc + torch.sum(preds == target.data)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):

            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            _, preds = torch.max(output, 1)
            loss = criterion(output, target)
            
            valid_acc = valid_acc + torch.sum(preds == target.data)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        train_acc = train_acc/len(loaders['train'].dataset)
        valid_acc = valid_acc/len(loaders['valid'].dataset)
        
        train_accuracy_list.append(train_acc)
        train_loss_list.append(train_loss)
        valid_accuracy_list.append(valid_acc)
        valid_loss_list.append(valid_loss)
        
        print('Epoch: {} \tTraining Acc: {:6f} \tTraining Loss: {:6f} \tValidation Acc: {:6f} \tValidation Loss: {:.6f}'.format(
            epoch,
            train_acc,
            train_loss,
            valid_acc,
            valid_loss
            ))

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss  
            
    return model

In [ ]:
model = train(n_epochs, loaders, model, optimizer, criterion, use_cuda, '/mnt/c/Users/sumit/Downloads/Compressed/model_final.pth')

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(train_loss_list, label="train_loss")
plt.title("Train-Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

In [ ]:
plt.style.use("ggplot")
plt.figure()

plt.plot(train_accuracy_list, label="train_acc")
plt.plot(valid_accuracy_list, label="valid_acc")

plt.title("Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

# Model Deployment Plan
Model Saving Objective: Save the best-performing model after training and hyperparameter tuning.

In [ ]:
torch.save(model, '/mnt/c/Users/sumit/Downloads/Compressed/models/model_final.pth')

# Deployment and Inference
Objective: The model can be deployed in real-world applications, such as monitoring forest fires. The model will be able to classify images from surveillance cameras in real-time.

In [ ]:
model = torch.load('/mnt/c/Users/sumit/Downloads/Compressed/Fire-Smoke-Detection-master/Fire-Smoke-Detection-master/trained-models/model_final.pth')
model.eval()

# Integration into a System
Objective: The model can be integrated into a larger system for real-time detection of fire, smoke, or non-fire. Action: Use a web or mobile app to send images to the model and retrieve classification results in real-time.

Summary of Tasks and Activities Collect the data: Download or gather the relevant dataset of images (fire, smoke, non-fire). Data preprocessing: Clean the data, remove corrupted images, and augment the dataset. Feature engineering: Extract features through CNN layers (no explicit manual feature extraction). Train/Test Split: Split the data into training and validation sets. Metrics for evaluation: Accuracy is selected as the evaluation metric. Model selection, training, and assessment: Build and train a CNN, then evaluate it on the test set. Hyperparameter tuning: Tune the model’s hyperparameters using Keras Tuner. Model deployment plan: Save the model and create a plan for real-time deployment and inference.

# Model Prediction
Objective: Use the trained model to make predictions on new (test) images.


In [ ]:
class_names = class_names = ['Fire', 'Neutral', 'Smoke']

def predict(image):
    prediction_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    image = prediction_transform(image)[:3,:,:].unsqueeze(0)
    image = image.cuda()

    pred = model(image)
    idx = torch.argmax(pred)
    prob = pred[0][idx].item()*100
    
    return class_names[idx], prob

# Image

In [ ]:
img = Image.open('/mnt/c/Users/sumit/Downloads/Compressed/FIRE-SMOKE-DATASET_2/FIRE-SMOKE-DATASET/Train/Neutral/image_41.jpg')
plt.imshow(img)
plt.show()

prediction, prob = predict(img)
print(prediction, prob)

In [ ]:
img = Image.open('/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/fire/Fire (45).jpeg')
plt.imshow(img)
plt.show()

prediction, prob = predict(img)
print(prediction, prob)

In [ ]:
img = Image.open('/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/Smoke/Smoke (945).jpg')
plt.imshow(img)
plt.show()

prediction, prob = predict(img)
print(prediction, prob)

# Video

In [ ]:
cap = cv2.VideoCapture('/mnt/c/Users/sumit/Downloads/Compressed/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/Smoke/videoplayback.mp4')

while True:

    ret, image = cap.read()
    draw = image.copy()
    draw = cv2.resize(draw,(640,480))
    
    draw = transforms.ToPILImage()(draw)
    prediction, prob = predict(draw)
    
    if prediction == 'Neutral':
        color = (0, 255, 0)
    else:
        color = (0, 0, 255)
    cv2.putText(image, (prediction+' '+str(prob)), (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    

    cv2.imshow('framename', image)
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
pip freeze > requirements.txt